In [1]:
import pyodbc
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
import random
import numpy as np
import tensorflow as tf

server = 'cyse492group8.database.windows.net'
database = 'CYSE492_Project'
username = 'cyseadmin'
password = 'GroupProject492'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

In [2]:
cursor.execute("SELECT [Control Identifier],[Control Text]  FROM [dbo].['NIST_800.53_controls$']")

labels = []
texts = []
for row in cursor:
    texts.append(row[1])
    if 'AC' in row[0]:
        labels.append(1)
    if 'AT' in row[0]:
        labels.append(2)
    if 'AU' in row[0]:
        labels.append(3)
    if 'CA' in row[0]:
        labels.append(4)
    if 'CM' in row[0]:
        labels.append(5)
    if 'CP' in row[0]:
        labels.append(6)
    if 'IA' in row[0]:
        labels.append(7)
    if 'IR' in row[0]:
        labels.append(8)
    if 'MA' in row[0]:
        labels.append(9)
    if 'MP' in row[0]:
        labels.append(10)
    if 'PE' in row[0]:
        labels.append(11)
    if 'PL' in row[0]:
        labels.append(12)
    if 'PM' in row[0]:
        labels.append(13)
    if 'PS' in row[0]:
        labels.append(14)
    if 'PT' in row[0]:
        labels.append(15)
    if 'RA' in row[0]:
        labels.append(16)
    if 'SA' in row[0]:
        labels.append(17)
    if 'SC' in row[0]:
        labels.append(18)
    if 'SI' in row[0]:
        labels.append(19)
    if 'SR' in row[0]:
        labels.append(20)

x_test = []
y_test = []
x_act = np.array(texts)
y_act = np.array(labels)

# test is the actual test, act is used for the training
for i in range(500):
    random_index = random.randrange(len(labels))
    x_test.append(texts.pop(random_index))
    y_test.append(labels.pop(random_index))
x_test = np.array(x_test)
y_test = np.array(y_test)

In [3]:
# Hyperparameters
max_words = 15000 # max number of words to use in the vocabulary was 15k
max_len = 400 # max length of each text (in terms of number of words) 250
embedding_dim = 400 # dimension of word embeddings was 100
lstm_units = 32 # number of units in the LSTM layer
# num_classes = len(set(labels)) + 1 # number of classes
num_classes = len(set(y_act)) + 1 # number of classes
#num_classes2 = len(set(y_test)) + 1

# Tokenize the texts and create a vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_act)

# Pad the sequences so they all have the same length
x = pad_sequences(sequences, maxlen=max_len)


# Create one-hot encoded labels
y = keras.utils.to_categorical(y_act, num_classes)

# y_test = keras.utils.to_categorical(y_test, num_classes)
# Build the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(lstm_units))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(x, y, batch_size=32, epochs=20)# was 32,20




Epoch 1/20


38/38 [==============================] - 21s 286ms/step - loss: 2.9241 - accuracy: 0.1329
Epoch 2/20
38/38 [==============================] - 10s 267ms/step - loss: 2.6088 - accuracy: 0.2649
Epoch 3/20
38/38 [==============================] - 10s 258ms/step - loss: 2.2130 - accuracy: 0.3701
Epoch 4/20
38/38 [==============================] - 10s 272ms/step - loss: 1.7609 - accuracy: 0.5290
Epoch 5/20
38/38 [==============================] - 10s 249ms/step - loss: 1.3872 - accuracy: 0.6526
Epoch 6/20
38/38 [==============================] - 11s 294ms/step - loss: 1.1013 - accuracy: 0.7620
Epoch 7/20
38/38 [==============================] - 11s 278ms/step - loss: 0.8488 - accuracy: 0.8318
Epoch 8/20
38/38 [==============================] - 10s 254ms/step - loss: 0.6577 - accuracy: 0.8839
Epoch 9/20
38/38 [==============================] - 10s 273ms/step - loss: 0.5091 - accuracy: 0.9142
Epoch 10/20
38/38 [==============================] - 10s 273ms/step - loss: 0.5786 - ac

In [4]:
###padding
## reuse old tokenizer for x_test
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_test)

## Pad the sequences so they all have the same length
x_test = pad_sequences(sequences, maxlen=max_len)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Test the model     
#If you want to test different x_test/y_test you can compile that section instead of retraining the whole thing again
accr = model.evaluate(x_test,y_test) #x and y instead of test?
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

#for i in range(10):
#    result = tf.argmax(model.predict(tf.expand_dims(texts[i], 0)), axis=1)
#    print(result.numpy(), labels[i])

16/16 [==============================] - 3s 69ms/step - loss: 0.0950 - accuracy: 0.9800
Test set
  Loss: 0.095
  Accuracy: 0.980


In [ ]:
print(x_test.shape)
print(y_test.shape)
print(x.shape)
print(y.shape)
print(x_test)
print(y_test)
print(x)
print(y)
print(x_act[1])

In [ ]:
print(len(set(y_act)))